In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
from IPython.display import display
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# 한글 폰트 설정
# 한글 폰트 설치 코드
!apt-get update -qq
!apt-get install fonts-nanum* -qq
# 설치 후 런타임 재시작 필요

Selecting previously unselected package fonts-nanum.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

NanumBarunGothic


In [ ]:
import pandas as pd
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표시
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/데이터분석 스터디/data/data_add_score.csv')
submission = pd.read_csv('/content/drive/MyDrive/데이터분석 스터디/data/sample_submission.csv')

data['출근자수'] = data['본사정원수'] - data['본사휴가자수'] - data['본사출장자수'] - data['현본사소속재택근무자수']
data['야근비율'] = data['본사시간외근무명령서승인건수'] / data['본사정원수']

train = data.iloc[:1204] 
test = data.iloc[1204:]

train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,break_score,lunch_score,dinner_score,출근자수,야근비율
0,2016-02-01,월,2601,50,150,238,0.000,1039.000,331.000,0,4,4,2401.000,0.092
1,2016-02-02,화,2601,50,173,319,0.000,867.000,560.000,0,12,8,2378.000,0.123
2,2016-02-03,수,2601,56,180,111,0.000,1017.000,573.000,0,5,4,2365.000,0.043
3,2016-02-04,목,2601,104,220,355,0.000,978.000,525.000,0,9,5,2277.000,0.136
4,2016-02-05,금,2601,278,181,34,0.000,925.000,330.000,0,5,12,2142.000,0.013


In [ ]:
# 변수 선택
x_train = train[['요일', '출근자수', '야근비율', 'lunch_score', 'dinner_score']]
x_test = test[['요일', '출근자수', '야근비율', 'lunch_score', 'dinner_score']]
x_train = pd.get_dummies(x_train, columns = ['요일'])
x_test = pd.get_dummies(x_test, columns = ['요일'])
#x_train['요일'] = x_train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
#x_test['요일'] = x_test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
y1_train = train['중식계']
y2_train = train['석식계']
y1_test = test['중식계']
y2_test = test['석식계']

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

### 1. 기반 모델 : 랜덤포레스트, GBRegressor, XGBoost, NGB Regressor 사용
### 2. 메타 모델 : XGBoost 사용

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from ngboost import NGBRegressor

# 랜덤포레스트
rf_lunch = RandomForestRegressor(n_estimators = 1500, max_features = 2)
rf_dinner = RandomForestRegressor(n_estimators = 500, max_features = 4)

# GBRegressor
gbr_lunch = GradientBoostingRegressor(n_estimators = 50, subsample = 0.1, 
                                      learning_rate = 0.01, max_depth = 8)
gbr_dinner = GradientBoostingRegressor(n_estimators = 500, subsample = 0.1,
                                       learning_rate = 0.01, max_depth = 10)

# XGBoost
xgb_lunch = XGBRegressor()
xgb_dinner = XGBRegressor()

# NGB Regressor
ngb_lunch = NGBRegressor()
ngb_dinner = NGBRegressor()

In [ ]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = x_train.values
X_test_n = x_test.values
y1_train_n = y1_train.values
y2_train_n = y2_train.values

In [ ]:
# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환

# 중식
rf_train_lunch, rf_test_lunch = get_stacking_base_datasets(rf_lunch, X_train_n, y1_train_n, X_test_n, 5)
gbr_train_lunch, gbr_test_lunch = get_stacking_base_datasets(gbr_lunch, X_train_n, y1_train_n, X_test_n, 5)
xgb_train_lunch, xgb_test_lunch = get_stacking_base_datasets(xgb_lunch, X_train_n, y1_train_n, X_test_n, 5)  
ngb_train_lunch, ngb_test_lunch = get_stacking_base_datasets(ngb_lunch, X_train_n, y1_train_n, X_test_n, 5)

# 석식
rf_train_dinner, rf_test_dinner = get_stacking_base_datasets(rf_dinner, X_train_n, y2_train_n, X_test_n, 5)
gbr_train_dinner, gbr_test_dinner = get_stacking_base_datasets(gbr_dinner, X_train_n, y2_train_n, X_test_n, 5)
xgb_train_dinner, xgb_test_dinner = get_stacking_base_datasets(xgb_dinner, X_train_n, y2_train_n, X_test_n, 5)  
ngb_train_dinner, ngb_test_dinner = get_stacking_base_datasets(ngb_dinner, X_train_n, y2_train_n, X_test_n, 5)

RandomForestRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
GradientBoostingRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[08:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[08:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[08:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[08:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  4  시작 
[08:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
NGBRegressor  mo

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  

# 중식
Stack_final_X_train_lunch = np.concatenate((rf_train_lunch, gbr_train_lunch, 
                                      xgb_train_lunch, ngb_train_lunch), axis=1)
Stack_final_X_test_lunch = np.concatenate((rf_test_lunch, gbr_test_lunch, 
                                     xgb_test_lunch, ngb_test_lunch), axis=1)

# 석식
Stack_final_X_train_dinner = np.concatenate((rf_train_dinner, gbr_train_dinner, 
                                      xgb_train_dinner, ngb_train_dinner), axis=1)
Stack_final_X_test_dinner = np.concatenate((rf_test_dinner, gbr_test_dinner, 
                                     xgb_test_dinner, ngb_test_dinner), axis=1)

In [ ]:
# 최종 메타 모델은 XGBoost 모델 적용
meta_model_xgb_lunch = XGBRegressor()
meta_model_xgb_dinner = XGBRegressor()

meta_model_xgb_lunch.fit(Stack_final_X_train_lunch, y1_train)
meta_model_xgb_dinner.fit(Stack_final_X_train_dinner, y2_train)

[08:13:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:13:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [ ]:
# 추론
pred1 = meta_model_xgb_lunch.predict(Stack_final_X_test_lunch)
pred2 = meta_model_xgb_dinner.predict(Stack_final_X_test_dinner)

In [ ]:
# 결과 제출
submission['중식계'] = pred1
submission['석식계'] = pred2

In [ ]:
submission.to_csv('/content/drive/MyDrive/데이터분석 스터디/data/stacking.csv', index=False)